In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

def get_data_loaders(batch_size=64, data_dir="data"):
    """Create and return train and test data loaders."""
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    # Download and load training data
    train_dataset = datasets.MNIST(
        root=data_dir,
        train=True,
        download=True,
        transform=transform
    )
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True
    )

    # Download and load test data
    test_dataset = datasets.MNIST(
        root=data_dir,
        train=False,
        download=True,
        transform=transform
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False
    )

    print(f"\nDataset sizes:")
    print(f"Training set: {len(train_dataset):,} images")
    print(f"Test set:     {len(test_dataset):,} images")
    print(f"Batch size:   {batch_size}")
    print(f"Training batches: {len(train_loader)}")
    print(f"Test batches:     {len(test_loader)}\n")

    return train_loader, test_loader 

In [2]:
import torch.nn as nn
import torch.nn.functional as F
class MNISTModel(nn.Module):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1, bias=False) #28
        self.norm1 = nn.BatchNorm2d(8)
        self.drop = nn.Dropout2d(0.1)
        self.conv2 = nn.Conv2d(8, 12, 3, padding=1, bias=False) #28
        self.norm2 = nn.BatchNorm2d(12)
        self.drop = nn.Dropout2d(0.1)
        self.pool = nn.MaxPool2d(2, 2) #14

        self.conv3 = nn.Conv2d(12, 16, 3, padding=1, bias=False) #14
        self.norm3 = nn.BatchNorm2d(16)
        self.drop = nn.Dropout2d(0.1)
        self.conv4 = nn.Conv2d(16, 20, 3, padding=1, bias=False) #14
        self.norm4 = nn.BatchNorm2d(20)
        self.drop = nn.Dropout2d(0.1)
        self.pool = nn.MaxPool2d(2,2) #7

        self.conv5 = nn.Conv2d(20, 24, 3, bias=False) #5
        self.norm5 = nn.BatchNorm2d(24)
        self.conv6 = nn.Conv2d(24, 28, 3, bias=False) #3
        self.antman = nn.Conv2d(28, 10 , 1, bias=False)#3
        self.gap = nn.AvgPool2d(3)#1

    def forward(self, x):
      x = F.relu(self.conv1(x))
      x = self.norm1(x)
      x = self.drop(x)
      x = F.relu(self.conv2(x))
      x = self.norm2(x)
      x = self.drop(x)
      x = self.pool(x)

      x = F.relu(self.conv3(x))
      x = self.norm3(x)
      x = self.drop(x)
      x = F.relu(self.conv4(x))
      x = self.norm4(x)
      x = self.drop(x)
      x = self.pool(x)

      x = F.relu(self.conv5(x))
      x = self.norm5(x)
      x = F.relu(self.conv6(x))
      x = self.antman(x)
      x = self.gap(x)
      x = x.view(-1, 10)

      return F.log_softmax(x, dim=1)

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import os

class Trainer:
    def __init__(self, model, train_loader, test_loader, 
                 learning_rate=0.001, device=None):
        self.device = device if device else torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = model.to(self.device)
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    def train_epoch(self):
        self.model.train()
        total_loss = 0
        correct = 0
        total = 0
        
        for batch_idx, (data, target) in enumerate(tqdm(self.train_loader, desc="Training")):
            data, target = data.to(self.device), target.to(self.device)
            self.optimizer.zero_grad()
            output = self.model(data)
            loss = self.criterion(output, target)
            loss.backward()
            self.optimizer.step()
            
            total_loss += loss.item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            total += target.size(0)
            
        avg_loss = total_loss / len(self.train_loader)
        accuracy = 100. * correct / total
        return avg_loss, accuracy

    def evaluate(self):
        self.model.eval()
        test_loss = 0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for data, target in tqdm(self.test_loader, desc="Testing "):
                data, target = data.to(self.device), target.to(self.device)
                output = self.model(data)
                test_loss += self.criterion(output, target).item()
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()
                total += target.size(0)

        avg_loss = test_loss / len(self.test_loader)
        accuracy = 100. * correct / total
        return avg_loss, accuracy

    def train(self, n_epochs=5, model_save_path="models/mnist_model.pth"):
        # Create models directory if it doesn't exist
        os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
        
        best_accuracy = 0.0
        print("\nStarting training...")
        print("-" * 60)
        
        for epoch in range(n_epochs):
            print(f"\nEpoch [{epoch+1}/{n_epochs}]")
            
            # Training phase
            train_loss, train_acc = self.train_epoch()
            
            # Testing phase
            test_loss, test_acc = self.evaluate()
            
            # Print metrics
            print("\nMetrics:")
            print(f"{'Train Loss':>12}: {train_loss:.4f}  {'Train Accuracy':>16}: {train_acc:.2f}%")
            print(f"{'Test Loss':>12}: {test_loss:.4f}  {'Test Accuracy':>16}: {test_acc:.2f}%")
            
            # Save best model
            if test_acc > best_accuracy:
                best_accuracy = test_acc
                torch.save(self.model.state_dict(), model_save_path)
                print(f"\n🔥 New best model saved! (Accuracy: {test_acc:.2f}%)")
            
            print("-" * 60)
        
        print(f"\nTraining completed! Best test accuracy: {best_accuracy:.2f}%")

In [4]:
import torch
# from src.dataset import get_data_loaders
# from src.model import MNISTModel
# from src.trainer import Trainer
# from src.utils import plot_prediction

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def main():
    # Set random seed for reproducibility
    torch.manual_seed(42)
    
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Get data loaders
    train_loader, test_loader = get_data_loaders(batch_size=64)
    
    # Initialize model
    model = MNISTModel()
    
    # Print model parameters
    num_params = count_parameters(model)
    print(f"\nModel Parameters: {num_params:,}")
    
    # Initialize trainer
    trainer = Trainer(
        model=model,
        train_loader=train_loader,
        test_loader=test_loader,
        learning_rate=0.001,
        device=device
    )
    
    # Train the model
    trainer.train(n_epochs=14)
    
    # Plot some predictions
    # plot_prediction(model, test_loader, device)

main()

Using device: cuda

Dataset sizes:
Training set: 60,000 images
Test set:     10,000 images
Batch size:   64
Training batches: 938
Test batches:     157


Model Parameters: 16,352

Starting training...
------------------------------------------------------------

Epoch [1/14]


Testing : 100%|███████████████████████████████████████████████████████████████████████████| 157/157 [00:02<00:00, 69.37it/s]



Metrics:
  Train Loss: 0.2935    Train Accuracy: 91.54%
   Test Loss: 0.0458     Test Accuracy: 98.54%

🔥 New best model saved! (Accuracy: 98.54%)
------------------------------------------------------------

Epoch [2/14]


Testing : 100%|███████████████████████████████████████████████████████████████████████████| 157/157 [00:02<00:00, 73.63it/s]



Metrics:
  Train Loss: 0.0694    Train Accuracy: 97.83%
   Test Loss: 0.0375     Test Accuracy: 98.83%

🔥 New best model saved! (Accuracy: 98.83%)
------------------------------------------------------------

Epoch [3/14]


...


Metrics:
  Train Loss: 0.0224    Train Accuracy: 99.24%
   Test Loss: 0.0196     Test Accuracy: 99.37%

🔥 New best model saved! (Accuracy: 99.37%)
------------------------------------------------------------

Epoch [13/14]


Testing : 100%|███████████████████████████████████████████████████████████████████████████| 157/157 [00:02<00:00, 72.71it/s]



Metrics:
  Train Loss: 0.0219    Train Accuracy: 99.32%
   Test Loss: 0.0201     Test Accuracy: 99.42%

🔥 New best model saved! (Accuracy: 99.42%)
------------------------------------------------------------

Epoch [14/14]


Testing : 100%|███████████████████████████████████████████████████████████████████████████| 157/157 [00:02<00:00, 73.69it/s]


Metrics:
  Train Loss: 0.0208    Train Accuracy: 99.33%
   Test Loss: 0.0187     Test Accuracy: 99.42%
------------------------------------------------------------

Training completed! Best test accuracy: 99.42%
